In [59]:
import scipy.stats as sts
import numpy as np
import pandas as pd
import plotly.express as px
np.random.seed(0)

# Estimators
def nonParametricOptimalQuantity(distributionData, tau):
    optimalQuantity = np.quantile(distributionData,tau)
    return optimalQuantity

def profit(tau, demand, quantity):
    price = 1
    cost = 1 - tau
    return price * np.minimum(demand,quantity) - cost * quantity

def parametricOptimalQuantity(estimators, tau):
    optimalQuantity = sts.norm.ppf(tau, loc = estimators[0], scale = estimators[1])
    return optimalQuantity

# Evaluation of test statistics functions
def empericalRootMeanSquaredError(m, optimalQuantities, tau, meanDemand, stdevDemand):
    realOptimalQuantity = parametricOptimalQuantity([meanDemand, stdevDemand], tau)
    rmse = np.sqrt(1/m * np.sum((optimalQuantities - realOptimalQuantity)**2))
    return rmse

def empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand):
    return empericalRootMeanSquaredError(m, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand) / empericalRootMeanSquaredError(m, parametricOptimalQuantitys, tau, meanDemand, stdevDemand)

def empericalServiceLevel(m, optimalQuantities, demand):
    indicatorFunction = np.where(optimalQuantities.T >= np.array(demand), 1, 0)
    return 1 / m * np.sum(indicatorFunction)

def empericalProfitLoss(m, optimalQuantities, demand, tau, meanDemand, stdevDemand):
    realOptimalQuantity = parametricOptimalQuantity([meanDemand, stdevDemand], tau)
    profitReal = profit(tau, meanDemand, realOptimalQuantity)
    profitEstimator = profit(tau, demand, optimalQuantities.T)
    return 1 / m * np.sum(np.abs((profitReal - profitEstimator) /  profitReal))

def empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand):
    return empericalProfitLoss(m, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand) / empericalProfitLoss(m, parametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)
def fit(feature):
    return np.mean(feature), np.std(feature)
def monteCarlo(m, tau, n, meanDemand, stdevDemand):
    # Define arrays
    realOptimalQuantity = np.empty((m,1))
    parametricOptimalQuantitys = np.empty((m,1))
    nonParametricOptimalQuantitys = np.empty((m,1))
    allParameters = np.empty((m,2))

    # Compute estimators and optimal quantities
    for j in range(m):
        distributionData = sts.norm.rvs(loc = meanDemand, scale = stdevDemand, size = n)
        allParameters[j] = parameters = fit(distributionData)
        parametricOptimalQuantitys[j] = parametricOptimalQuantity(parameters, tau)
        nonParametricOptimalQuantitys[j] = nonParametricOptimalQuantity(distributionData, tau)
        realOptimalQuantity[j] = parametricOptimalQuantity([meanDemand,stdevDemand], tau)
    
    demand = allParameters[:,0] # in case of normal

    # Compute evaluation statistics
    rmse = empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand)
    eslParametric = empericalServiceLevel(m, parametricOptimalQuantitys, demand)
    eslNonParametric = empericalServiceLevel(m, nonParametricOptimalQuantitys, demand)
    eplr = empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)
    result = {
                    'MonteCarlo iterations' : m,
                    'Sample Size': n,
                    'Target Service Level': tau,
                    'Param Values': np.mean(allParameters, axis=0),
                    'real optimal quantity' : np.mean(realOptimalQuantity),
                    'Parm optimal quantity': np.mean(parametricOptimalQuantitys),
                    'nonParm optimal quantity': np.mean(nonParametricOptimalQuantitys),
                    'RMSE Ratio': rmse,
                    'SL nonParam': eslParametric,
                    'SL Param': eslNonParametric,
                    'EPLR': eplr
                }
    return result





In [60]:
numberOfMontecarloIterations = 1000
stdevDemand = 15
meanDemand = 120

tauArray = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
nArray = [10, 50, 100, 200]

results = []
for tau in tauArray:
    for n in nArray:
        result = monteCarlo(numberOfMontecarloIterations, tau, n, meanDemand, stdevDemand)
        results.append(result)


In [61]:
df = pd.DataFrame(results)
df

,MonteCarlo iterations,Sample Size,Target Service Level,Param Values,real optimal quantity,Parm optimal quantity,nonParm optimal quantity,RMSE Ratio,SL nonParam,SL Param,EPLR
0,1000,10,0.01,"[119.72349419762591, 13.720362743553597]",85.104782,87.805157,97.520275,1.644325,0.0,0.000,1.771051
1,1000,50,0.01,"[120.0140967807625, 14.740757309842326]",85.104782,85.721967,88.993291,1.640041,0.0,0.000,1.683588
2,1000,100,0.01,"[120.14130607280387, 14.858540982582157]",85.104782,85.575171,88.044106,1.871786,0.0,0.000,1.941332
3,1000,200,0.01,"[119.98878184228278, 14.947526742899928]",85.104782,85.215635,86.417555,1.903595,0.0,0.000,1.944169
4,1000,10,0.05,"[120.11252657015035, 13.742808256866427]",95.327196,97.507619,100.920980,1.207038,0.0,0.000,1.243371
5,1000,50,0.05,"[120.13138641676703, 14.761916927166336]",95.327196,95.850194,96.874366,1.303617,0.0,0.000,1.301021
6,1000,100,0.05,"[120.02817687096912, 14.90756341535131]",95.327196,95.507417,96.083780,1.299937,0.0,0.000,1.305279
7,1000,200,0.05,"[120.0361744493198, 14.940970788928059]",95.327196,95.460464,95.627141,1.360500,0.0,0.000,1.377528
8,1000,10,0.10,"[119.83957962745934, 13.861372332356554]",100.776727,102.075516,103.998818,1.184618,0.0,0.000,1.204583
9,1000,50,0.10,"[119.98330792635866, 14.757411232445705]",100.776727,101.070924,101.435344,1.251779,0.0,0.000,1.261884


In [62]:
fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher RMSE means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/RMSE-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()

fig = px.scatter(df, x='Target Service Level', y='EPLR', color="Sample Size", title="TSL vs EPLR, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher EPLR means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/EPLR-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()

#fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, higher RMSE means parametric performs better")
#fig.show()


In [63]:
numberOfMontecarloIterations = 1000
stdevDemand = 20
meanDemand = 100

tauArray = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
nArray = [10, 50, 100, 200]

results = []
for tau in tauArray:
    for n in nArray:
        result = monteCarlo(numberOfMontecarloIterations, tau, n, meanDemand, stdevDemand)
        results.append(result)


In [64]:
df = pd.DataFrame(results)
df

,MonteCarlo iterations,Sample Size,Target Service Level,Param Values,real optimal quantity,Parm optimal quantity,nonParm optimal quantity,RMSE Ratio,SL nonParam,SL Param,EPLR
0,1000,10,0.01,"[100.05276221108602, 18.322430190433316]",53.473043,57.428416,70.081064,1.612756,0.0,0.000,1.746746
1,1000,50,0.01,"[99.86599774239826, 19.625352516938257]",53.473043,54.210601,58.687906,1.620333,0.0,0.000,1.697920
2,1000,100,0.01,"[99.98793184995574, 19.825710639774723]",53.473043,53.866432,56.936919,1.855067,0.0,0.000,1.909708
3,1000,200,0.01,"[100.01753715293296, 19.93699120840811]",53.473043,53.637160,55.492622,1.876973,0.0,0.000,1.934502
4,1000,10,0.05,"[100.17412834628907, 18.583272451373194]",67.102927,69.607365,74.013078,1.181105,0.0,0.000,1.212582
5,1000,50,0.05,"[100.01290058587084, 19.74449497854405]",67.102927,67.536096,68.846935,1.301301,0.0,0.000,1.307192
6,1000,100,0.05,"[100.13613182339489, 19.7881913695077]",67.102927,67.587453,68.364898,1.318215,0.0,0.000,1.319447
7,1000,200,0.05,"[100.05332981205532, 19.88140049309111]",67.102927,67.351336,67.659238,1.303488,0.0,0.000,1.289564
8,1000,10,0.10,"[100.23942617137259, 18.565601623716454]",74.368969,76.446650,78.927581,1.183254,0.0,0.000,1.187543
9,1000,50,0.10,"[100.04874979208519, 19.739044117267085]",74.368969,74.752147,75.468564,1.272098,0.0,0.000,1.274230


In [65]:
fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher RMSE means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/RMSE-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()

fig = px.scatter(df, x='Target Service Level', y='EPLR', color="Sample Size", title="TSL vs EPLR, mean: {0} | stddev: {1} | iter. {2}<br><sup>higher EPLR means parametric performs better</sup>".format(meanDemand, stdevDemand, numberOfMontecarloIterations))
fig.update_layout(font_size = 15)
#fig.write_image("Excercise part 3/EPLR-{0}-{1}.pdf".format(meanDemand, stdevDemand))
fig.show()